# Objective

To explore and visualize the Cars4U dataset, build a linear regression model to predict the prices of used cars, and generate a set of insights and recommendations that will help the business.


# Problem Definition

How do we define a good predictive pricing model that can effectively predict the price of used cars and can help the business in devising profitable strategies using differential pricing? And, What does the performance assessment look like for such a model?


# Key Questions

* How many types of car brands and models are available?
* How does the Price vary across different car brands ?
* How does year of manufacture vary with the pricing of used cars?
* High performance cars are good for certain types of customers. How many cars are available for different brands with 200 bhp and above?
* What percentage of cars have a high bhp across different brands?
* How does the New_Price vary with used car price across all brands? How do we determine depreciation?
* How does the number of seats vary with price across different brands?



According to https://www.monash.edu/, Differential pricing is a pricing strategy in which a company sets different prices for the same product on the basis of differing customer type, time of purchase, etc; also called Discriminatory Pricing, Flexible Pricing, Multiple Pricing, Variable Pricing.

In the case of Cars4U, we will use a combination of dependent variables to come up with a pricing model for used cars and test it to ascertain that it can meet the differential pricing requirement.

# Import all the necessary libraries

In [4]:
%load_ext nb_black
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import datetime

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [5]:
# Load the data set into a pandas dataframe and preview it
data = pd.read_csv("./data_sets/used_cars_data.csv")


<IPython.core.display.Javascript object>

# Data background and contents

In [6]:
# checking the shape of the data
print(f"There are {data.shape[0]} rows and {data.shape[1]} columns.")

There are 7253 rows and 14 columns.


<IPython.core.display.Javascript object>

In [ ]:
# Data sample
data.sample(10, random_state=2)

,S.No.,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
4584,4584,Tata Tigor 1.05 Revotorq XT,Kochi,2018,28973,Diesel,Manual,First,24.7 kmpl,1047 CC,69 bhp,5.0,7.15,6.64
6505,6505,Volkswagen Vento Diesel Highline,Chennai,2011,76041,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,13.23,NaN
3675,3675,Maruti Swift VDI,Ahmedabad,2012,65000,Diesel,Manual,First,22.9 kmpl,1248 CC,74 bhp,5.0,7.88,3.90
5654,5654,Hyundai i20 Magna Optional 1.2,Kochi,2014,42315,Petrol,Manual,First,18.5 kmpl,1197 CC,82.9 bhp,5.0,10.25,4.79
4297,4297,Toyota Camry 2.5 G,Mumbai,2014,68400,Petrol,Automatic,First,12.98 kmpl,2494 CC,178.4 bhp,5.0,40.62,13.50
2603,2603,Mercedes-Benz New C-Class 220 CDI AT,Jaipur,2010,74213,Diesel,Automatic,First,14.84 kmpl,2143 CC,170 bhp,5.0,49.49,11.50
4337,4337,Volkswagen Vento Petrol Highline AT,Kochi,2014,32283,Petrol,Automatic,Second,14.4 kmpl,1598 CC,103.6 bhp,5.0,13.23,5.41
6625,6625,Maruti Swift VDI BSIV,Kolkata,2012,72000,Diesel,Manual,First,17.8 kmpl,1248 CC,NaN,5.0,7.88,NaN
2846,2846,Skoda Superb Elegance 1.8 TSI AT,Kochi,2011,73783,Petrol,Automatic,Second,13.7 kmpl,1798 CC,157.75 bhp,5.0,33.36,5.23
1237,1237,Audi Q3 2.0 TDI Quattro,Hyderabad,2013,60000,Diesel,Automatic,First,17.32 kmpl,1968 CC,184 bhp,5.0,44.80,21.00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Looking at the data, we do not need S.no (Serial number) so we will drop it right away

In [ ]:
data.drop(["S.No."], axis=1, inplace=True)
# reset the indexes
data.reset_index()

In [ ]:
# checking column datatypes and number of non-null values
data.info()

* We have 3 floats, 3 Integers and & columns of Object type

In [ ]:
# Check for missing values
data.isnull().sum()

 We have missing values in the data on Engine, Power, Seats and Price columns

In [ ]:
# Check for duplicated values
data.duplicated().sum()
data.loc[data.duplicated()]

We have 1 duplicate in the data. Let us treat it accordingly 

In [ ]:
data = data.drop_duplicates()

In [ ]:
#verify if duplicates dropped
data.duplicated().sum()

In [ ]:
# Let's look at the statistical summary of the data
data.describe(include="all").T

* There are 2041 different car brands in the data
* We see that Mahindra XUV500 W8 2WD is the most common car
* Majority of the cars are sort from Mumbai
* The average year of manufacture is 2013 with the oldest car manufactured in 1996 and newest is 2019
* The average kilometers driven is 58699 with min & max of 171 and 6500000 KMS. The max kilometers driven value seems like an outlier, we will investigate it
* Most common type of fuel type is Diesel
* Majority of the cars sold are of Manual transmission
* Majority of the cars sold are new given that owner type is First meaning that most customers prefer new cars as 1st choice preference
* The modal mileage is 17.0 kmpl.
* The modal power is 74 bhp. We observe units in the column value hence the need to separate to obtain min,max std for this column
* The maximum price for new cars is 375 INR while the minimum price use 3.91 INR Lakhs
* The average price of used cars is 9.5 INR , minimum oprice is 0.44 INR Lakhs while max price is 160. The deviation from mean is high so this could be an outlier
* We observe that 50% of all new cars cost 11 INR and above
* We observe that 50% of all used cars cost 5.64 INR and above
* Most of the cars are 5 seaters



# Exploratory Data Analysis

## Univariate Analysis

In [ ]:
# function to plot a boxplot and a histogram along the same scale.
# Credit : From Anime Rating Case study


def histogram_boxplot(data, feature, figsize=(12, 7), kde=False, bins=None, hue=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to the show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet", hue=hue
    )  # boxplot will be created and a star will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins, palette="winter", hue=hue
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram
    
    # function to create labeled barplots


def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 1, 5))
    else:
        plt.figure(figsize=(n + 1, 5))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n].sort_values(),
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

### Year

In [ ]:
histogram_boxplot(data, "Year")

* The distrubution for year is left Skewed
* We observe outliers in this column
* We can use Year to determing car age which will be more useful

### Kilometers_Driven 

In [ ]:
histogram_boxplot(data, "Kilometers_Driven", bins=10000)

* The distribution for Kilometers_Driven is left skewed.
* We have one outlier. Needs further investigation

#### Investigating the outlier in kilometers driven

In [ ]:
data.loc[data["Kilometers_Driven"] > 6000000]

We conclude that it is not normal for a car to have driven 6.5 m kilomters.  This outlier needs to be treated accordingly 

### Seats

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data=data, x="Seats")
plt.show()

* The distribution for this column is randomly distributed. Bivariate analysis will help to get more insights

### Price

In [ ]:
histogram_boxplot(data, "Price")

* The distribution for this column skewed to the left
* We also have many outliers for this column hence the need for proper treatment

### Location

In [ ]:
labeled_barplot(data, "Location")

* We observe that majority of the car locations are available at Mumbai, Hyderabad, Combatore, Koci and Pune


### Fuel Type

In [ ]:
labeled_barplot(data, "Fuel_Type")

* Diesel and Petrol Engine cars form the majority of the cars in the data-set

### Transmission

In [ ]:
labeled_barplot(data, "Transmission")

* We have 2 tyeps of car transmission in the dataset
* Manual cars form the majority in the dataset

### Owner_Type

In [ ]:
labeled_barplot(data, "Owner_Type")

* First Owner Type form the majorty of cars in the data set. 
* Fourth & Above owner types are the least sold cars

## Bivariate Analysis

### Heat Map

In [ ]:
plt.figure(figsize=(12, 7))
sns.heatmap(data.corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral")
plt.show()

### Pair plot

In [ ]:
plt.figure(figsize=(12, 7))
sns.pairplot(data=data, hue="Owner_Type")
plt.show()

* We observe a strong correlation between the used car Price and the price of a new car
* A negative correlation is observed between Price and Kilometers_Driven which is normal. Cars with more KMs tend to have a lower price
* Price has a positive correlation with Year. Wich is normal because the newer the car the higher the price.
* Price has a weak postitve corelation with Seats. 



### Price Vs New Price vs Owner Type

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(data=data, x="Price", y="New_Price", hue="Owner_Type")
plt.show()

### Price Vs kilometers driven

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(data=data, x="Kilometers_Driven", y="Price", hue="Transmission")
plt.show()


* We observe a strong correlation between new and old car pricers especially from first car owner types than other owner types.
* Customers tend to prefer cars with lower kilometers driven and Manual transmission type
* Prices for cars with manual transmission cars are rerelatively higher than prices for cars with automatic transmission
* Automatic cars are more expensive than manual cars

# Data Preprocessing

In [ ]:
# Check the data
data.head()

From the data we observe that Mileage, Engine and power have units. Let us strip the units and convert the columns to numericals and replace null values with na

#### Remove units from Engine

In [ ]:
# Striping units from Engine column
data["Engine"] = data["Engine"].str.rstrip(" CC")
# Replace null with nan
data["Engine"] = data["Engine"].replace(regex="null", value=np.nan)

#### Remove units from  power

In [ ]:
# Striping units from Power column
data["Power"] = data["Power"].str.rstrip(" bhp")
# Replace null with nan
data["Power"] = data["Power"].replace(regex="null", value=np.nan)

#### Remove units from Mileage

In [ ]:
# Striping units from Mileage column and replace
data["Mileage"] = data["Mileage"].str.rstrip(" kmpl")
data["Mileage"] = data["Mileage"].str.rstrip(" km/kg")
data["Mileage"] = data["Mileage"].replace(regex="null", value=np.nan)

In [ ]:
# Verify the data
data.sample(10)

In [ ]:
# Convert to numerical columns
data["Mileage"] = data["Mileage"].astype(float)
data["Power"] = data["Power"].astype(float)
data["Engine"] = data["Engine"].astype(float)

In [ ]:
# Verify data types
data.info()

## Outlier Treatment

* We will investigate and treat outliers in Price , Kilometers Driven & Power columns

In [ ]:
#Function to plot box & hist plots
def boxHistplot(data, feature):
    plt.figure(figsize=(10,5))
    sns.histplot(data=data,x=feature)
    plt.show()
    
    plt.figure(figsize=(10,5))
    sns.boxplot(data=data,x=feature)
    plt.show()
    
def densityPlot(data,feature):
    plt.figure(figsize=(10,5))
    sns.distplot(data[feature], kde=True)
    plt.show()

### Power Outlier Treatment

In [ ]:
boxHistplot(data,'Power')
densityPlot(data,'Power')

* The box plot flags many values as outliers but the histplot indicates left right skewness. These points seem consistent with the overall distribution  

In [ ]:
# Examinining sensitivity of the outliers by checking how many values are greater than 4*IQR

In [ ]:
def outlierSensitivity(data, feature):
    quartiles = np.quantile(data[feature][data[feature].notnull()], [.25, .75])
    power_iqr = 4 * (quartiles[1] - quartiles[0])
    print(f'Q1 = {quartiles[0]}, Q3 = {quartiles[1]}, IQR = {power_iqr}')
    outlier_powers = data.loc[np.abs(data['Power'] - data[feature].median()) > power_iqr, feature]
    print(f'We have {outlier_powers.shape[0]} values on {feature} above IQR')
    



In [ ]:
outlierSensitivity(data,'Power')

* The distirbution is fairly normal with mild right skewness. All data points seem valid. We will ignore Power treatment for now

### Kilometers Driven Outlier Treatment

In [ ]:
boxHistplot(data,'Kilometers_Driven')
densityPlot(data,'Kilometers_Driven')

* We seem to have one extreme outlier in this column let us investigate

In [ ]:
data[data['Kilometers_Driven'] > 1000000]

* It is not normal for a car to have more than 6.5 m. We will replace this outlier with na and replace with median value

In [ ]:
#Replace extreme outlier with nan
data.loc[2328, "Kilometers_Driven"] = np.nan

In [ ]:
#Inpute with median
data["Kilometers_Driven"].fillna(data["Kilometers_Driven"].median(), inplace=True)  # median imputation for Kilometers Drive

* We still have some outliers in KM_Driven but distribution has improved.
* We can use log transformation to further treat this column

In [ ]:
#Log transformation
data['Kilometers_Driven' + '_log'] = np.log(data['Kilometers_Driven'])

In [ ]:
#verify after transformation
boxHistplot(data,'Kilometers_Driven_log')
densityPlot(data,'Kilometers_Driven_log')

### Price Outlier Treatment

In [ ]:
# Preview distribution
boxHistplot(data,'Price')
densityPlot(data,'Price')

* We observe a number of columns, one extreme outlier with 160 needs further investigation

In [ ]:
data.loc[data['Price'] > 120]

* Price column has an outlier of 160. Since we have missing values on this column, we will log transform it later after missing value treatment

## Missing Value Treatment

In [ ]:
data.isnull().sum()

Since these are numerical values, we will inpute missing values with the mean, median accordingly

### Mileage Treatment

In [ ]:
data["Mileage"].fillna(
    data["Mileage"].median(), inplace=True
)  # median imputation for Mileage

### Power Treatment

In [ ]:
# Check if we have zero values in power and replace with nan for missing values treatment
data.query("Power == 0")

In [ ]:
data["Power"].fillna(
    data["Power"].median(), inplace=True
)  # median imputation for power

### Engine Treatment

In [ ]:
# Check if we have zero values in engine and replace with nan for missing values treatment
data.query("Engine == 0")

In [ ]:
data["Engine"].fillna(
    data["Engine"].median(), inplace=True
)  # mean imputation for Engine

### Seats Treatment

In [ ]:
# Check if we have zero values in seats and replace with nan for missing values treatment
data.query("Seats == 0")

In [ ]:
# Replace with Nan
data.loc[3999, "Seats"] = np.nan

In [ ]:
data["Seats"].fillna(data["Seats"].mean(), inplace=True)  # mean imputation for seats

### Price Treatment

In [ ]:
# Check if we have zero values in price and replace with nan for missing values treatment
data.query("Price == 0")

* Since this is the target variable, we should drop all missing values

In [ ]:
data=data.dropna(subset=['Price'])

### Verify Missing values result

In [ ]:
# Verfy missing values
data.isnull().sum()

## Feature Engineering

#### Year

In [ ]:
# processing to determine age of car. We will use 2023 as current year
data["Year_Current"] = 2023
data["Car_Age"] = data["Year_Current"] - data["Year"]
# Drop Column Year_Current as we will no longer need it
data.drop("Year_Current", axis=1, inplace=True)

In [ ]:
#### Analysis of Car Age

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=data,x='Car_Age')
plt.show()

* Car_Age is right skewed. We can apply log transformation

In [ ]:
data['Car_Age_log'] = np.log(data['Car_Age'])

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=data,x='Car_Age_log')
plt.show()

#### Fuel Type

In [ ]:
# Checking unique Fuel Types
data["Fuel_Type"].unique()

    We observe 4 fuel types in the dataset. No further action here

#### Price

In [ ]:
# Check  if we have zero values
data.query("Price == 0")

#### Name

In [ ]:
def retrieve_brand(name):
    brand = name.split(' ')[0]
    return brand

def retrieve_model(name):
    model = name.split(' ')[1]
    return model

Company brand affects the price of cars.  
* Since the Name combines both brand & model. We will strip and have brand/Model columns suing functions created earlier

In [ ]:
data["Brand"] = data["Name"].apply(retrieve_brand)
data["Model"] = data["Name"].apply(retrieve_model)

In [ ]:
# Preview Brands
data["Brand"].unique()

In [ ]:
# Preview Models
data["Model"].unique()

* We observe some names have spilled over to Model column like Land Rover where Land is in Brand and Rover is in model. Same case for mini cooper This needs to be corrected
* We observe duplicate names for Isuzu in lower and upper case, probably a data entry error 

In [ ]:
#Check how many rows have ISUZU
data.loc[data.Brand == 'ISUZU','Brand'].shape


In [ ]:
# Convert to lower case
data.loc[data.Brand == "ISUZU", "Brand"] = "Isuzu"

In [ ]:
# Convert Land Brand Name to Land Rover and Mini to Mini Cooper
data.loc[data.Brand == "Mini", "Brand"] = "Mini_Cooper"
data.loc[data.Brand == "Land", "Brand"] = "Land_Rover"

In [ ]:
# Verify the corrections
data.loc[(data.Brand == "Mini_Cooper") | (data.Brand == "Land_Rover"), "Brand"].sample(
    10
)

In [ ]:
#  model & name columns
data.drop("Name", axis=1, inplace=True)

In [ ]:
# Verify the data
data.sample(10)

In [ ]:
## Drop Model Column

#### Processing New Price

In [ ]:
data['New_Price_log']= np.log(data['New_Price'])

#### Processing Price

In [ ]:
data['Price_log'] = np.log(data['Price'])

## Answers to key questions

### Q. How many types of car brands and models are available?

In [ ]:
print(
    f"There are {data['Brand'].nunique()}  and  {data['Model'].nunique()} brands  in the data set."
)

In [ ]:
labeled_barplot(data, "Brand")
data.drop('Model',axis=1, inplace=True)

 Maruti, Hyundai cars have the highest number of cars in the dataset

### How does the Price vary across different car brands ?

#### Analyzing with price for new Cars

In [ ]:
data.groupby("Brand")["New_Price"].mean().sort_values(ascending=False)

#### Analyzing with price for used Cars

In [ ]:
data.groupby("Brand")["Price"].mean().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.barplot(data=data, y="New_Price", x="Brand")
plt.xticks(rotation=90)
plt.title("New Cars")

plt.subplot(1, 2, 2)
sns.barplot(data=data, y="Price", x="Brand")
plt.xticks(rotation=90)
plt.title("Used Cars")

plt.show()

* Amongst new cars, Bently is the most expensive while lamborghini is the most expensive car brand amongst used cars
* Maruti, Hyundai , Honda , audi, Nissan, Toyota, Volkswagen, Tata, Renault, Mahindra ,Ford, Datsun, Fiat, Hindustan, OpeCorsa , Isuzu, Force seem competitively priced for new and used cars 

### How does year of manufacture vary with the pricing of used cars?

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.barplot(data=data, y="New_Price", x="Year")
plt.xticks(rotation=90)
plt.title("New Cars")

plt.subplot(1, 2, 2)
sns.barplot(data=data, y="Price", x="Year")
plt.xticks(rotation=90)
plt.title("Used Cars")

plt.show()

#Drop Year Column
data.drop('Year',axis=1,inplace=True)

* The price of cars has remained unchanged for new cars manufactured over the period from 1996 to 2019
* On the other hand, the price for used cars  increases with age and is left skewed

### High performance cars are good for certain types of customers. How many cars are available for different brands with 200 bhp and above?

In [ ]:
print(f"We have {data.loc[data['Power'] > 200].shape[0]} cars with more that 200bhp")

### What percentage of cars have a high bhp across different models/brands?

In [ ]:
data_high = data.loc[data['Power'] > 200]
(data_high.Brand.value_counts() / data.Brand.value_counts()).sort_values(ascending=False)

* Bentley, Lamborghini have the hightest bhp followed by Porsche, Jaguar, Mercedez

 ### How does the New_Price vary with used car price across all brands?

In [ ]:


data_price_comparision = data.groupby(["Brand"])["New_Price", "Price"].mean()
data_price_comparision["Depreciation"] = (
    (data_price_comparision.New_Price - data_price_comparision.Price)
    / data_price_comparision.New_Price
) * 100



In [ ]:
data_price_comparision.sort_values(ascending=False, by="Depreciation")

* Ambassador, Bentley, Smart, Chevrolet cars brand has the highest depreciation rate and lowest resale value
* Jeep and Force cars have lowest depreciation and highest resale value
* Lamborghini resale price is high


### How does the number of seats vary with the prices across different brands?

In [ ]:
plt.figure(figsize=(15, 5))


plt.subplot(1, 2, 2)
sns.barplot(data=data, y="Price", x="Seats")
plt.xticks(rotation=90)
plt.title("Used Car Price Vs Seats")

plt.show()

* 2 Seater cars are the most expensive 
* Seats 5.280494581828286 seem odd. We need to analyze and treat accoridngly


#### Seats treatment for erroneous value

In [ ]:
#Fetch all Seats with 5.2** and round to 5 
indexes_seats = data.loc[data["Seats"] == 5.280494581828286].index

In [ ]:
for index in indexes_seats:
    data.loc[index,'Seats'] = 5

In [ ]:
#Verify if seat value is treated

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 2)
sns.barplot(data=data, y="Price", x="Seats")
plt.xticks(rotation=90)
plt.title("Used Cars vs Seats")

plt.show()

In [ ]:
# Sample check to confirm 
data.sample(10)

In [ ]:
## Recheck duplicates and drop them
data = data.drop_duplicates()

#### Statistical summary after data processing

In [ ]:
data.describe(include="all").T

In [ ]:
#Review correlation between variables after data processing
plt.figure(figsize=(12, 7))
sns.heatmap(data.corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral")
plt.show()

* Average car age is approximately 10 years while the Minimum car age is 4 while max is 27 years
* We observe that average mileage is 18 kmpl
* The average engine capacity from the dataset is now visible at 1616 cc with maximum Engine Capcity at 5998
* The average power is 113 bhp and maximum power is 616 bhp
* The mean car age is approximately 10 years
* We observe a strong correlation between the used car Price and Power

## Drop original transformed columns

In [ ]:
data.info()

In [ ]:
data.drop(['Kilometers_Driven','Car_Age','New_Price'],axis=1,inplace=True)



In [ ]:
data.head()

# Model Building

In [ ]:
# create function to convert object columns to dummies
def convertObjectColumnsToDummies(X):
   
    X = pd.get_dummies(
        X,
        columns=data.select_dtypes(include=["object"]).columns.tolist(),
        drop_first=True,
    )
    X= X.rename(columns={'Owner_Type_Fourth & Above' : 'Owner_Type_Fourth_above','Brand_Mercedes-Benz' :'Brand_Mercedez_Benz' },inplace=True)
    return X

In [ ]:
data.head()

## Define dependent and independent variables

In [ ]:
X = data.drop(["Price", "Price_log"], axis=1)

# Let us create 2 target variables y & y2 for Price and Price_log respectively
y = data[["Price"]]
y2 = data[["Price_log"]]

## Creating dummy variables

In [ ]:
X = convertObjectColumnsToDummies(X)

In [ ]:
#Verify Independent Variables
X.head()

## Split Data into train and test for 1st Target Variable - Price

In [ ]:
X_Train, X_Test, y_train ,y_test = train_test_split(X, y, test_size=0.3,random_state=42)

## Split Data into train and test for 2nd Target Variable Price_log

In [ ]:
X_Train, X_Test, y_train2 ,y_test2 = train_test_split(X, y2, test_size=0.3,random_state=42)

## Fit the linear Model

In [ ]:
# Function to fit the model
def fitModel(train_x,train_y):
    linear_regression_model = LinearRegression()
    linear_regression_model.fit(train_x,train_y)
    


In [ ]:
X_Train

In [ ]:
# Fit y
fitModel(X_Train,y_train)

In [ ]:
# Fit y2
fitModel(X_Train,y_train2)

## Checking Model Performance

* We will use RMSE ,MAE , MAPE and R2 from sklearn library to assess performance

In [ ]:
#Function to compute  adjusted R2
def _adjustedRScore(independentVariables,dependentVariable,prediction):
    r_squared = r2_score(dependentVariable,prediction)
    n = independentVariables.shape[0]
    k = independentVariables.shape[1]
    
    return  1 - ((1 - r_squared) * (n - 1) / (n - k - 1))



# function to compute R2, RMSE, MAR for performance check of a regression model
def model_performance_regression(model, independentVariables, dependentVariable):

    
    prediction = model.predict(independentVariables)

    r_squared_score = r2_score(dependentVariable, prediction)  
    adjusted_rsquared_score = _adjustedRScore(independentVariables, dependentVariable, prediction)  
    root_mean_squared_error = np.sqrt(mse(dependentVariable, prediction))  
    mean_absolute_error = mae(dependentVariable, prediction)  
    mean_absolute_percentage_error = mean_absolute_percentage_error_score(dependentVariable,prediction)
  

    # create metrics DF
    data_frame_performance = pd.DataFrame(
        {
            "RMSE": root_mean_squared_error,
            "MAE": mean_absolute_error,
            "R-squared": r_squared_score,
            "Adj. R-squared": adjusted_rsquared_score,
            "Mean Absolute Error %": mean_absolute_percentage_error
          
        },
        index=[0],
    )

    return data_frame_performance

# function to compute Mean Absolute Error %
def mean_absolute_percentage_error_score(dependentVariable, prediction):
    return np.mean(np.abs(dependentVariable - prediction) / dependentVariable, axis=0) * 100


###  Model Performance on Training Data for first Target Variable y -Price

In [ ]:
print("Test Performance on Training Data - y \n")
regression_model_training_performance = model_performance_regression(linear_regression_model, X_Train, y_train)
regression_model_training_performance

###  Model Performance on Test  Data for first target variable y -Price

In [ ]:
print("Test Performance on Test Data y\n")
regression_model_test_performance = model_performance_regression(linear_regression_model, X_Test, y_test)
regression_model_test_performance

###  Model Performance on Training Data for second Target Variable y2 - Price_log

In [ ]:
print("Test Performance on Training Data - y2 \n")
regression_model_training_performance = model_performance_regression(linear_regression_model, X_Train, y_train2)
regression_model_training_performance

###  Model Performance on Test Data for second Target Variable y2 - Price_log

In [ ]:
print("Test Performance on Test Data y2\n")
regression_model_test_performance = model_performance_regression(linear_regression_model, X_Test, y_test2)
regression_model_test_performance

**Model Observations**

- The train and test $R^2$ are 0.77 and 0.82, indicating that the model explains 77% and 82% of the total variation in the train and test sets respectively. 
- RMSE values on the train and test sets are comparable.
- MAE indicates that our current model is able to predict Price of used cars within a mean error of 1.47 on the test set and RMSE of 3.15 on the testing set
- The model performs poorly on Price_log y2 with an MAE of 9.3 R2 of -185 and RMSE of 11
- The model performs better on Price y and  can be used to predict Price of cars with MAE of 2, RMSE 4.45 with R-Squared 0.82 

# Conclusions & recommendations

- We have been able to build a predictive model that can be used by car to predict the prices of used cares with an $R^2$ of 0.72 on the training set.


- Cars4U can use this model to predict the prices of used cars within a mean error of 1.65 on the test set


- Cars4U can also explore other machine learning models which might be able to better predict the prices of used cars 

- Cars4U can focus on cars with less depreciation and high resale value

- First owner type cars tend to sell alot due to lower kilometers driven and tend to be newer that other owner types. 

- Older cars tend to have a lower price than newer cars, the business can focus on relatively new cars due to the high demand




